In [ ]:
# import os

# # Set the environment variable to a new cache location
# # The default cache space is too small
# os.environ['HF_HOME'] = '/home/ec2-user/SageMaker'

In [1]:
%%capture

! pip install torch datasets trl bitsandbytes accelerate peft

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer
import torch
import re

In [3]:
%%time

from transformers import BitsAndBytesConfig, AutoModelForCausalLM

model_name = "Qwen/Qwen2.5-7B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_storage=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

CPU times: user 20.7 s, sys: 21.8 s, total: 42.5 s
Wall time: 1min 26s


In [4]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear",
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 80,740,352 || all params: 7,696,356,864 || trainable%: 1.0491


In [5]:
# gsm8k = load_dataset("openai/gsm8k", "main")
gsm8k = load_dataset("eagle0504/openai-gsm8k-enhanced-using-together-ai-deepseek-train8k-test1k-v1")
train_ds = gsm8k["train"]   # 7,473 training examples
test_ds = gsm8k["test"]     # 1,319 test examples for evaluationtest_ds = gsm8k["test"]     # 1,319 test examples for evaluation

README.md:   0%|          | 0.00/2.89k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.35M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/417k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [6]:
train_ds

Dataset({
    features: ['question', 'answer', 'cot'],
    num_rows: 7473
})

In [17]:
from datasets import Dataset

# Assuming train_ds is already loaded and is a Dataset object
def add_prompt(example):
    # Create a new entry in the dictionary with key 'prompt' copying 'question'
    example["prompt"] = example["question"]
    return example

# Apply the function to each example in the dataset
train_ds = train_ds.map(add_prompt)

# Check the updated dataset structure
print(train_ds)

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'cot', 'prompt'],
    num_rows: 7473
})


In [48]:
from trl import GRPOTrainer, GRPOConfig
from datasets import load_dataset


# 2. Define a simple reward function
def reward_func(completions, **kwargs):
    """Example: Reward longer completions"""
    return [float(len(completion)) for completion in completions]


# 3. Configure training
training_args = GRPOConfig(
    output_dir="output",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    logging_steps=10,
    report_to="none"  # Disable integration with any external reporting tools like wandb
)

# 4. Initialize and train
trainer = GRPOTrainer(
    model=model,  # e.g. "Qwen/Qwen2-0.5B-Instruct"
    args=training_args,
    train_dataset=train_ds,
    reward_funcs=reward_func,
)

In [ ]:
%%time

trainer.train()